In [6]:
import pickle
import pandas as pd

In [7]:
import os

In [8]:
# If you want to see debug messages turn this on. Re-start the kernel to change the debugging
import logging
logging.basicConfig(level=logging.INFO)

In [9]:
import value_aggregation as pm

In [ ]:
# NB: The authors of the Kaleido paper personally gave us access to their data

In [10]:
os.listdir("../../external_data/kaleido/")

['social_chem.pkl', 'commonense_morality.pkl', 'sbic.pkl', 'moral_stories.pkl']

In [11]:
# With all of these, only count it is a disagreement if neither theory has a tie

### Loading in the Kaleido data and putting it in our framework

In [13]:
for file in os.listdir('../../external_data/kaleido/'):
    print(f'file: {file}')
    
    data = pd.read_pickle(f'../../external_data/kaleido/{file}')
    scenarios = list(data.keys())
    disagreements = {}
    actions = ['supports', 'opposes', 'either']

    for scenario in scenarios:
        scenario_df = data[scenario]

        if scenario_df.empty:
            continue

        outcomes = {}
        beliefs = {}

        for index, row in scenario_df.iterrows():
            beliefs[row['value']] = row['relevant']

            for action in actions:
                if action not in outcomes:
                    outcomes[action] = {}
                outcomes[action][row['value']] = row[action]


        gameState = pm.VoteGameState(beliefs, outcomes)

        mec_decision = pm.run_mec(gameState, none_if_tie=True)
        nbs_decision = pm.run_nash_bargain(gameState, none_if_tie=True)

        if mec_decision != nbs_decision:
            disagreements[scenario] = gameState

    print(f'# disagreements: {len(disagreements)}')
    print(f'# scenarios: {len(scenarios)}')
    print(f'% disagree: {len(disagreements) / len(scenarios)}')
    print()


file: social_chem.pkl
# disagreements: 1251
# scenarios: 80254
% disagree: 0.015588008074363894

file: commonense_morality.pkl
# disagreements: 35
# scenarios: 3408
% disagree: 0.010269953051643193

file: sbic.pkl
# disagreements: 48
# scenarios: 2960
% disagree: 0.016216216216216217

file: moral_stories.pkl
# disagreements: 187
# scenarios: 12000
% disagree: 0.015583333333333333



### Drilling down on an example in data

In [42]:
scenario = list(disagreements.keys())[6]
print(scenario)
print(pm.run_mec(disagreements[scenario], none_if_tie=True))
print(pm.run_nash_bargain(disagreements[scenario], none_if_tie=True))

If Owen's friend Jake is having trouble with his girlfriend, and Owen wants to help Jake feel better, is Owen sending flowers to Jake's girlfriend in Jake's name, hoping to patch things up moral?
opposes
supports


In [ ]:
{"beliefs" : {"Duty to protect Jake's emotional well-being": 0.9939894080162048,
              "Jake's girlfriend's right to not be deceived": 0.9939202070236206,
              'Friendship/loyalty': 0.9850156903266907,
              "Duty to respect the boundaries of others' relationships": 0.9793990254402161,
              'Emotional well-being': 0.9613832831382751,
               'Duty to help friends in need': 0.9584091305732727,
               'Right to emotional support': 0.9294604659080505,
               'Autonomy': 0.8827911019325256,
               'Compassion': 0.8633553385734558,
               'Right to privacy': 0.8515092730522156,
               'Honesty': 0.8409119844436646},
"outcomes" : {'supports': 
                    {"Duty to protect Jake's emotional well-being": 1.6430970951914787,
                   "Jake's girlfriend's right to not be deceived": 1.0,
                   'Friendship/loyalty': 1.738052198663354,
                   "Duty to respect the boundaries of others' relationships": 1.0,
                   'Emotional well-being': 1.8114324077032506,
                   'Duty to help friends in need': 1.9162462325766683,
                   'Right to emotional support': 1.5669531486928463,
                   'Autonomy': 1.0,
                   'Compassion': 1.9201710536144674,
                   'Right to privacy': 1.0,
                   'Honesty': 1.0},
               'opposes': 
                   {"Duty to protect Jake's emotional well-being": 1.0,
                   "Jake's girlfriend's right to not be deceived": 1.9759761763270944,
                   'Friendship/loyalty': 1.0,
                   "Duty to respect the boundaries of others' relationships": 1.9664880065247416,
                   'Emotional well-being': 1.0,
                   'Duty to help friends in need': 1.0,
                   'Right to emotional support': 1.0,
                   'Autonomy': 1.9628316410817206,
                   'Compassion': 1.0,
                   'Right to privacy': 1.975601093377918,
                   'Honesty': 1.9307380486279726},
               'either': 
                    {"Duty to protect Jake's emotional well-being": 1.239181987941265,
                   "Jake's girlfriend's right to not be deceived": 1.0140749865677208,
                   'Friendship/loyalty': 1.2352443132549524,
                   "Duty to respect the boundaries of others' relationships": 1.0182461319491267,
                   'Emotional well-being': 1.170298622455448,
                   'Duty to help friends in need': 1.0707711474969983,
                   'Right to emotional support': 1.315317865461111,
                   'Autonomy': 1.0231415447779,
                   'Compassion': 1.0659999488852918,
                   'Right to privacy': 1.0170531100593507,
                   'Honesty': 1.04481471888721}
                }
}

In [19]:
# drilling down on a specific scenario
data[scenarios[1]]

,action,vrd,value,relevant,supports,opposes,either
0,In the context that Jordan woke up very early ...,Duty,Duty to care for one's own health and well-being,0.976914,0.987893,0.000192,0.011915
1,In the context that Jordan woke up very early ...,Duty,Duty to respect others' sleep schedules,0.975380,0.346086,0.134612,0.519302
2,In the context that Jordan woke up very early ...,Value,Self-care,0.946529,0.983202,0.000465,0.016333
4,In the context that Jordan woke up very early ...,Value,Convenience,0.915920,0.965940,0.003734,0.030327
6,In the context that Jordan woke up very early ...,Right,Autonomy,0.876289,0.902547,0.001103,0.096350
7,In the context that Jordan woke up very early ...,Value,Consideration for others,0.876127,0.453058,0.076815,0.470127
8,In the context that Jordan woke up very early ...,Value,Autonomy,0.791178,0.904109,0.001385,0.094507
